In [1]:
import trafilatura
import requests
import lxml
from elasticsearch import Elasticsearch, helpers


In [2]:
vietnamnet = {
    "life": ["https://vietnamnet.vn/vn/doi-song/gia-dinh/",
             "https://vietnamnet.vn/vn/doi-song/chuyen-la/",
             "https://vietnamnet.vn/vn/doi-song/gioi-tre/",
             "https://vietnamnet.vn/vn/doi-song/tam-su/",
             "https://vietnamnet.vn/vn/doi-song/am-thuc/",
             "https://vietnamnet.vn/vn/doi-song/meo-vat/"],
    "news": ["https://vietnamnet.vn/vn/thoi-su/chong-tham-nhung/",
             "https://vietnamnet.vn/vn/thoi-su/quoc-hoi-khoa-xv/",
             "https://vietnamnet.vn/vn/thoi-su/an-toan-giao-thong/"
             "https://vietnamnet.vn/vn/thoi-su/moi-truong/",
             "https://vietnamnet.vn/vn/thoi-su/clip-nong/",
             "https://vietnamnet.vn/vn/thoi-su/bhxh-bhyt/",
             "https://vietnamnet.vn/vn/thoi-su/quoc-phong/"],
    "world": ["https://vietnamnet.vn/vn/the-gioi/binh-luan-quoc-te/",
              "https://vietnamnet.vn/vn/the-gioi/chan-dung/",
              "https://vietnamnet.vn/vn/the-gioi/ho-so/",
              "https://vietnamnet.vn/vn/the-gioi/the-gioi-do-day/",
              "https://vietnamnet.vn/vn/the-gioi/viet-nam-va-the-gioi/",
              "https://vietnamnet.vn/vn/the-gioi/quan-su/"]
}


In [9]:
vietnamnetLinks = {
    "life": [],
    "news": [],
    "world": []
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
for key, value in vietnamnet.items():
    for url in value:
        response = requests.get(url, headers=headers)
        tree = lxml.html.fromstring(response.content)
        title_elem = tree.cssselect('a.title')
        for elem in title_elem:
            vietnamnetLinks[key].append("https://vietnamnet.vn"+elem.get('href'))

print(len(vietnamnetLinks['life'])+len(vietnamnetLinks['news'])+len(vietnamnetLinks['world']))

783


In [8]:
vietnamnetContent = {
    "life": [],
    "news": [],
    "world": []
}
for key, value in vietnamnetLinks.items():
    for url in value:
        if(url.startswith('https://vietnamnet.vn/')):
            id = url.split('-')[-1].replace('.html', '')
            downloaded = trafilatura.fetch_url(url)
            tree = lxml.html.fromstring(str(downloaded))
            date_elem = len(tree.cssselect("div.breadcrumb-box__time p span")) and tree.cssselect("div.breadcrumb-box__time p span")[0].text or ""
            content = trafilatura.extract(downloaded)
            content = content.replace('\n', ' ')
            item = {"domain": "https://vietnamnet.vn/", "URL": url,
                    "content": content, "date": date_elem, "id": id}
            vietnamnetContent[key].append(item)

print(len(vietnamnetContent['life'])+len(vietnamnetContent['news'])+len(vietnamnetContent['world']))
print(vietnamnetContent['life'][0])

728834
816312
814528
828148
807124
810382
814281
827868
827879
827931
827475


KeyboardInterrupt: 

In [ ]:
client = Elasticsearch("http://localhost:9200")

client.info()


In [ ]:
def gendata():
    for key, value in vietnamnetContent.items():
        for item in value:
            yield {
                "_index": "journal",
                "_type": key,
                "_id": item['id'],
                "doc": item
            }


helpers.bulk(es, gendata())
